In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('glassdoor_jobs.csv')
df

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"See Tickets, TicketWeb, Vendini"
952,952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,-1
953,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1
954,954,Data Engineer,-1,Loading...\n\nTitle: Data Engineer\n\nLocation...,4.8,IGNW\n4.8,"Austin, TX","Portland, OR",201 to 500 employees,2015,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),Slalom


In [3]:
df.isna().sum()

Unnamed: 0           0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
dtype: int64

## TO DO
* salary parsing
* company name ---- text only
* state field
* age of company
* parsing of job description ----- skills --- python, R, etc

In [4]:
df.dtypes

Unnamed: 0             int64
Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Headquarters          object
Size                  object
Founded                int64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
Competitors           object
dtype: object

In [5]:
df.drop(columns = 'Unnamed: 0', inplace = True)

In [6]:
df = df[df['Salary Estimate'] != '-1'].reset_index().drop(columns = 'index')

In [7]:
df['Salary Estimate'][0]

'$53K-$91K (Glassdoor est.)'

In [8]:
salary= df['Salary Estimate'].apply(lambda x: x.split('(')[0])
salary.head()

0     $53K-$91K 
1    $63K-$112K 
2     $80K-$90K 
3     $56K-$97K 
4    $86K-$143K 
Name: Salary Estimate, dtype: object

In [9]:
minus_eks = salary.apply(lambda x: 
                        x.replace('Employer Provided Salary:', '').replace('K','').replace('$', '')).str.strip()

### Assumptions
* 8 working hours in a day
* 250 working days in a year

In [10]:
def remove_per_hour(x):
    if len(x) > 7:
        x = x.split(' ')[0]
        x = x.split('-')
        x0 = int(int(x[0])*8*250/1000)
        x1 = int(int(x[1])*8*250/1000)
        return str(x0)+'-'+str(x1)
    else:
        return x

In [11]:
remove_per_hour('24-39 Per Hour')

'48-78'

In [12]:
minus_ph = minus_eks.apply(lambda x: remove_per_hour(x))

In [13]:
def average(x):
    x = x.split('-')
    return np.average([int(x[0]), int(x[1])])

In [14]:
df['average_salary (K$)'] = minus_ph.apply(lambda x: average(x))

In [15]:
df['Company Name'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0
                                                                          else x['Company Name'][:-3], axis = 1)

In [16]:
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])

In [17]:
df['same_state'] = df.apply(lambda x: 1 if x['Location'] == x['Headquarters'] else 0, axis = 1)

In [18]:
df['Company_age (years)'] = df['Founded'].apply(lambda x: x if x < -1 else 2022 - x)

In [19]:
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [20]:
df['R'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

In [21]:
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

In [22]:
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

In [23]:
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

In [24]:
(df['python'].sum(),
df['R'].sum(),
df['aws'].sum(),
df['spark'].sum(),
df['excel'].sum())

(392, 2, 176, 167, 388)

In [25]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'average_salary (K$)', 'job_state', 'same_state', 'Company_age (years)',
       'python', 'R', 'aws', 'spark', 'excel'],
      dtype='object')

In [26]:
df['Revenue']

0             $50 to $100 million (USD)
1                $2 to $5 billion (USD)
2            $100 to $500 million (USD)
3      $500 million to $1 billion (USD)
4              Unknown / Non-Applicable
                     ...               
737                  $10+ billion (USD)
738          $100 to $500 million (USD)
739            Unknown / Non-Applicable
740             $5 to $10 million (USD)
741           $50 to $100 million (USD)
Name: Revenue, Length: 742, dtype: object

In [27]:
minus_usd = df['Revenue'].apply(lambda x: x.split('(')[0])

In [28]:
minus_usd

0             $50 to $100 million 
1                $2 to $5 billion 
2            $100 to $500 million 
3      $500 million to $1 billion 
4         Unknown / Non-Applicable
                  ...             
737                  $10+ billion 
738          $100 to $500 million 
739       Unknown / Non-Applicable
740             $5 to $10 million 
741           $50 to $100 million 
Name: Revenue, Length: 742, dtype: object

In [29]:
minus_unknown = minus_usd.apply(lambda x: 0 if x == 'Unknown / Non-Applicable' else x)

In [30]:
minus_unknown

0             $50 to $100 million 
1                $2 to $5 billion 
2            $100 to $500 million 
3      $500 million to $1 billion 
4                                0
                  ...             
737                  $10+ billion 
738          $100 to $500 million 
739                              0
740             $5 to $10 million 
741           $50 to $100 million 
Name: Revenue, Length: 742, dtype: object

1 billion = 1000 million

In [31]:
def get_revenue(x):
    x = x.split('to')
    
    if len(x) == 2:
        x0 = x[0].split(' ')[0][1:]
        x01 = x[0].split(' ')[1]

        x1 = x[1][1:].split(' ')[0][1:]
        x11 = x[1][1:].split(' ')[1]

        if x01 == 'billion' and x11 == 'billion':
            avg = 1000*(int(x0) + int(x1))/2

        elif x01 == '' and x11 == 'billion':
            avg = 1000*(int(x0) + int(x1))/2

        elif x01 == 'million' and x11 == 'billion':
            avg = (int(x0) + 1000*int(x1))/2

        else:
            avg = (int(x0)+int(x1))/2

        return avg
    
    else:
        return x[0]

In [32]:
getting_avg_revenue = minus_unknown.apply(lambda x: 0 if x == 0 else get_revenue(x))

In [33]:
for i in getting_avg_revenue:
    if len(str(i)) > 7:
        print(i)

$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
Less than $1 million 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
Less than $1 million 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
$10+ billion 
Less than $1 million 
Less than 

In [34]:
minus_10_plus_billion = getting_avg_revenue.apply(lambda x: 10*1000 if x == '$10+ billion ' else x)

In [35]:
minus_less_than_1_million = minus_10_plus_billion.apply(lambda x: 1 if x == 'Less than $1 million ' else x)

In [36]:
df['Average_Revenue (million $)'] = minus_less_than_1_million.apply(lambda x: float(x))

In [37]:
df['Average_Revenue (million $)'] = df['Average_Revenue (million $)'].apply(lambda x: 1 if x == 0 else x)

In [38]:
minus_employees = df['Size'].apply(lambda x: x.replace('employees', '')).str.strip()

In [39]:
minus_unknown = minus_employees.apply(lambda x: '-1' if x == 'Unknown' else x)

In [40]:
minus_unknown

0       501 to 1000
1            10000+
2       501 to 1000
3      1001 to 5000
4         51 to 200
           ...     
737          10000+
738    1001 to 5000
739     501 to 1000
740         1 to 50
741     501 to 1000
Name: Size, Length: 742, dtype: object

In [41]:
def get_avg_size(x):
    x = x.split('to')
    if len(x) == 2:
        avg = int( ( int(x[0]) + int(x[1]) ) / 2 )
        return avg
    elif x[0] == '-1':
        return 0
    else:
        return int( x[0][:-1] )

In [42]:
getting_avg_size = minus_unknown.apply(lambda x: get_avg_size(x))

In [43]:
s = 0
c = 0
for i in getting_avg_size:
    if i != 10000:
        s += i
        c += 1
avg = int(s/c)
avg

1918

In [44]:
df['Average_company_size'] = getting_avg_size.apply(lambda x: avg if x == 0 else x)

In [45]:
df['Type of ownership'].value_counts()

Company - Private                 410
Company - Public                  193
Nonprofit Organization             55
Subsidiary or Business Segment     34
Government                         15
Hospital                           15
College / University               13
Other Organization                  3
School / School District            2
Unknown                             1
-1                                  1
Name: Type of ownership, dtype: int64

In [46]:
df['Type of ownership'] = df['Type of ownership'].apply(lambda x: 'Other' if x != 'Company - Private' and x != 'Company - Public' else x)
df['Type of ownership'] = df['Type of ownership'].apply(lambda x: 'Private' if x == 'Company - Private' else x)
df['Type of ownership'] = df['Type of ownership'].apply(lambda x: 'Public' if x == 'Company - Public' else x)

In [47]:
df['Type of ownership'].value_counts()

Private    410
Public     193
Other      139
Name: Type of ownership, dtype: int64

In [48]:
df.dtypes

Job Title                       object
Salary Estimate                 object
Job Description                 object
Rating                         float64
Company Name                    object
Location                        object
Headquarters                    object
Size                            object
Founded                          int64
Type of ownership               object
Industry                        object
Sector                          object
Revenue                         object
Competitors                     object
average_salary (K$)            float64
job_state                       object
same_state                       int64
Company_age (years)              int64
python                           int64
R                                int64
aws                              int64
spark                            int64
excel                            int64
Average_Revenue (million $)    float64
Average_company_size             int64
dtype: object

In [49]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,job_state,same_state,Company_age (years),python,R,aws,spark,excel,Average_Revenue (million $),Average_company_size
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Private,...,NM,0,49,1,0,0,0,1,75.0,750
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other,...,MD,0,38,1,0,0,0,0,3500.0,10000
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Private,...,FL,1,12,1,0,0,1,1,300.0,750
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Other,...,WA,1,57,1,0,0,0,0,750.0,3000
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n,"New York, NY","New York, NY",51 to 200 employees,1998,Private,...,NY,1,24,1,0,0,0,1,1.0,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Public,...,MA,0,192,0,0,1,0,0,10000.0,10000
738,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Public,...,TN,0,16,1,0,1,1,0,300.0,3000
739,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,Other,...,PA,1,38,0,0,0,0,1,1.0,750
740,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Private,...,PA,0,2023,0,0,0,0,1,7.5,25


In [50]:
df.to_csv('salary_data_(cleaned).csv', index = False)